<a href="https://colab.research.google.com/github/Vikas-KM/tensorflow-learning/blob/master/Exercise_4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator



#### Wget for Anaconda Installation
- https://anaconda.org/menpo/wget

In [11]:

DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>DESIRED_ACCURACY):
      print('Reached 90% accuracy so cancelling')
      self.model.stop_training = True

callbacks = myCallback()

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/happy-or-sad.zip'

In [2]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=RMSprop(learning_rate=0.001),
    metrics=['accuracy'],
)

In [3]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(300,300),
        batch_size=10,
        class_mode='binary',        
      )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [4]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
      train_generator,
      epochs=15,
      steps_per_epoch=8,
      verbose=1,
      callbacks=[callbacks]
    )
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 7s 53ms/step - loss: 16.1699 - accuracy: 0.5859
Epoch 2/15
8/8 [==============================] - 0s 51ms/step - loss: 0.9257 - accuracy: 0.6226
Epoch 3/15
8/8 [==============================] - 0s 52ms/step - loss: 0.6678 - accuracy: 0.6548
Epoch 4/15
8/8 [==============================] - 0s 53ms/step - loss: 0.2443 - accuracy: 0.8910
Epoch 5/15
8/8 [==============================] - 0s 52ms/step - loss: 0.0730 - accuracy: 0.9684
Epoch 6/15
8/8 [==============================] - 0s 51ms/step - loss: 0.0273 - accuracy: 0.9944
Epoch 7/15
8/8 [==============================] - 0s 51ms/step - loss: 0.9212 - accuracy: 0.9070
Epoch 8/15
8/8 [==============================] - 0s 51ms/step - loss: 0.0522 - accuracy: 1.0000
Reached 90% accuracy so cancelling


### choose batch_size and steps_per_epoch such that they are close to the number of dataset
#### Example here we have 80 images so if we select batch_size of 8 then steps_per_epoch will be 10